In [1]:
!pip install farm-haystack[inference]

In [2]:
!pip install wikipedia-api

In [3]:
from wikipediaapi import Wikipedia
import wikipediaapi
import os

In [4]:
!unzip roman_empire.zip

"unzip" non � riconosciuto come comando interno o esterno,
 un programma eseguibile o un file batch.


In [5]:
from haystack.document_stores import InMemoryDocumentStore
from haystack.pipelines import Pipeline, ExtractiveQAPipeline
from haystack.nodes import DensePassageRetriever, TextConverter, PreProcessor, FARMReader
from haystack.utils import print_answers

c:\Users\AlessandroLaVeglia\anaconda3\envs\terronslayers\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
doc_store = InMemoryDocumentStore()

In [7]:
retriever = DensePassageRetriever(document_store=doc_store)

c:\Users\AlessandroLaVeglia\anaconda3\envs\terronslayers\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [8]:
text_converter = TextConverter()

In [9]:
preprocessor = PreProcessor(
    split_length=200,
    split_overlap=100
)

In [10]:
pipe = Pipeline()

In [11]:
pipe.add_node(component=text_converter, name="converter", inputs=["File"])
pipe.add_node(component=preprocessor, name="preprocessor", inputs=["converter"])
pipe.add_node(component=doc_store, name="store", inputs=["preprocessor"])

In [12]:
folder_name = "data\DnD Classes"
file_paths = [os.path.join(folder_name, filename) for filename in os.listdir(folder_name)]

pipe.run(file_paths=file_paths)

FileNotFoundError: [WinError 3] Impossibile trovare il percorso specificato: 'roman_empire'

In [ ]:
for doc in doc_store.get_all_documents_generator():
    print(doc)
    break

<Document: id=1d1625192fe6e5e8d9886e9ea552f4ff, content='Class - Fighter

Level 1 - Fighting Style: You adopt a particular style of fighting as your specialt...'>


In [ ]:
from haystack import document_stores
len(doc_store.get_all_documents())

59

In [ ]:
doc_store.update_embeddings(retriever=retriever)

Documents Processed: 10000 docs [00:12, 786.58 docs/s]        


In [ ]:
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", return_no_answer=True, top_k=100)

In [ ]:
pipe_qa = ExtractiveQAPipeline(reader=reader, retriever=retriever)

In [ ]:
ans = pipe_qa.run(query="How much damage does the eldritch blast do?")

Inferencing Samples: 100%|██████████| 1/1 [00:02<00:00,  2.95s/ Batches]


In [ ]:
print_answers(ans, details="medium")

'Query: How much damage does the eldritch blast do?'
'Answers:'
[   {'answer': '', 'score': 0.6542633351230939},
    {   'answer': '1d10 force damage',
        'context': 'ranged spell attack against the target. On a hit, the '
                   'target takes 1d10 force damage.\n'
                   '\n'
                   'At Higher Levels. The spell creates more than one beam '
                   'when you',
        'score': 0.6238809823989868},
    {   'answer': '1d6',
        'context': "At Higher Levels: The spell's damage increases by 1d6 when "
                   'you reach 5th level (2d6), 11th level (3d6), and 17th '
                   'level (4d8).\n'
                   'Infestation - Classes: D',
        'score': 0.46346092224121094},
    {   'answer': '1d8',
        'context': 'nd of your next turn.\n'
                   '\n'
                   'At Higher Levels. This spell’s damage increases by 1d8 '
                   'when you reach 5th level (2d8), 11th level (3d8), an